# PCA: finding out which features are the most important

In [2]:
import pandas as pd

## Part 1: Importing the data

### Importing cancer data

In [6]:
cancer_data = pd.read_csv("../data_clean/CDC_CancerByCounty/incidencerates/cancer_by_type.csv")
cancer_data.head()

,locale,fips,met_health_obj,incidence rate_per_100000,incidence rate_lower_95_confidence,incidence rate_upper_95_confidence,annual_count_avg,recent_trend_str,trend_last_5,trend_last_5_lower_95_confidence,...,cancer,file_name,race,sex,source_url,stage,stateFIPS,type,state,cancer_description
0,"US (SEER+NPCR)(1,10)",0.0,***,448.0,447.7,448.4,1638110.0,falling,-0.9,-1.6,...,1,incidencerates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,29,incd,missouri,All Cancer Sites
1,"US (SEER+NPCR)(1,10)",0.0,***,11.7,11.7,11.8,43864,stable,0.5,-0.7,...,3,incidencerates_016.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,29,incd,missouri,Oral Cavity & Pharynx
2,"US (SEER+NPCR)(1,10)",0.0,***,4.5,4.5,4.6,17084,stable,-0.8,-1.6,...,17,incidencerates_009.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,29,incd,missouri,Esophagus
3,"US (SEER+NPCR)(1,10)",0.0,***,6.6,6.5,6.6,23871,falling,-1.3,-2.0,...,18,incidencerates_020.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,29,incd,missouri,Stomach
4,"US (SEER+NPCR)(1,10)",0.0,Yes,38.7,38.6,38.8,140982,falling,-1.3,-2.1,...,20,incidencerates_008.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,29,incd,missouri,Colon & Rectum


In [7]:
cancer_data.columns

Index(['locale', 'fips', 'met_health_obj', 'incidence rate_per_100000',
       'incidence rate_lower_95_confidence',
       'incidence rate_upper_95_confidence', 'annual_count_avg',
       'recent_trend_str', 'trend_last_5', 'trend_last_5_lower_95_confidence',
       'trend_last_5_upper_95_confidence', 'age', 'areatype', 'cancer',
       'file_name', 'race', 'sex', 'source_url', 'stage', 'stateFIPS', 'type',
       'state', 'cancer_description'],
      dtype='object')

In [21]:
cancer_data = cancer_data[['locale', 'fips', 'annual_count_avg', 'areatype', 'cancer', 'stateFIPS', 'type',
       'state', 'cancer_description']]
cancer_data.head()

,locale,fips,annual_count_avg,areatype,cancer,stateFIPS,type,state,cancer_description
0,"US (SEER+NPCR)(1,10)",0.0,1638110.0,country,1,29,incd,missouri,All Cancer Sites
1,"US (SEER+NPCR)(1,10)",0.0,43864,country,3,29,incd,missouri,Oral Cavity & Pharynx
2,"US (SEER+NPCR)(1,10)",0.0,17084,country,17,29,incd,missouri,Esophagus
3,"US (SEER+NPCR)(1,10)",0.0,23871,country,18,29,incd,missouri,Stomach
4,"US (SEER+NPCR)(1,10)",0.0,140982,country,20,29,incd,missouri,Colon & Rectum


In [22]:
cancer_data.tail()

,locale,fips,annual_count_avg,areatype,cancer,stateFIPS,type,state,cancer_description
52656,"Weston County(6,10)",56045.0,3 or fewer,county,86,56,incd,wyoming,Non-Hodgkin Lymphoma
52657,"Weston County(6,10)",56045.0,3 or fewer,county,90,56,incd,wyoming,Leukemia
52658,"Weston County(6,10)",56045.0,3 or fewer,county,400,56,incd,wyoming,Breast (in situ) (Female)
52659,"Weston County(6,10)",56045.0,3 or fewer,county,515,56,incd,wyoming,"Childhood (Ages <20, All Sites)"
52660,"Weston County(6,10)",56045.0,3 or fewer,county,516,56,incd,wyoming,"Childhood (Ages <15, All Sites)"


### Importing industry data per county

In [9]:
industry_data = pd.read_csv("../data_clean/indicators_per-industry_per-county.csv")
industry_data.head()

,Unnamed: 0,fips,county,name,industry_code,industry_detail,relevant_naics,year,payann,estab,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,0,8001,1,"Adams County, Colorado",113000,timber and raw forest products,[113],2012,0,1,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,1,8001,1,"Adams County, Colorado",115000,agriculture and forestry support,[115],2012,290,6,...,0.00008,0.0,3.405586,0.0,0.000183,0.008769,0.95877,2.011564,211.423463,15.653502
2,2,8001,1,"Adams County, Colorado",211000,unrefined oil and gas,[211],2012,0,6,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,3,8001,1,"Adams County, Colorado",2122a0,"iron, gold, silver, and other metal ores","[21221, 21222, 21229]",2012,0,1,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,4,8001,1,"Adams County, Colorado",212310,dimensional stone,[21231],2012,0,4,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [15]:
industry_data.drop('Unnamed: 0', axis=1, inplace=True)

In [17]:
industry_data.head()

,fips,county,name,industry_code,industry_detail,relevant_naics,year,payann,estab,emp,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,8001,1,"Adams County, Colorado",113000,timber and raw forest products,[113],2012,0,1,0,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,8001,1,"Adams County, Colorado",115000,agriculture and forestry support,[115],2012,290,6,10,...,0.00008,0.0,3.405586,0.0,0.000183,0.008769,0.95877,2.011564,211.423463,15.653502
2,8001,1,"Adams County, Colorado",211000,unrefined oil and gas,[211],2012,0,6,0,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,8001,1,"Adams County, Colorado",2122a0,"iron, gold, silver, and other metal ores","[21221, 21222, 21229]",2012,0,1,0,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,8001,1,"Adams County, Colorado",212310,dimensional stone,[21231],2012,0,4,0,...,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [18]:
industry_data.columns

Index(['fips', 'county', 'name', 'industry_code', 'industry_detail',
       'relevant_naics', 'year', 'payann', 'estab', 'emp', 'ACID', 'ENRG',
       'ETOX', 'EUTR', 'FOOD', 'GCC', 'HAPS', 'HAZW', 'HC', 'HNC', 'HRSP',
       'HTOX', 'JOBS', 'LAND', 'METL', 'MINE', 'MSW', 'NREN', 'OZON', 'PEST',
       'REN', 'SMOG', 'VADD', 'WATR'],
      dtype='object')